In [10]:
%pylab inline

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense
from keras.layers import BatchNormalization, Dropout
from keras.layers import Activation

from tools import load_household_power_consumption
from tools import to_timeseries
from tools import split_train_test

Populating the interactive namespace from numpy and matplotlib


# Configuration

In [2]:
BATCH = 32

# Data

In [19]:
dataset, data_x, data_y = load_household_power_consumption()
train_x, train_y, test_x, test_y = split_train_test(to_timeseries(data_x, t=BATCH), 
                                                    to_timeseries(data_y, t=BATCH), 
                                                    train_ratio=0.8)
print('data_x:', data_x.shape)
print('data_y:', data_y.shape)
print('train_x:', train_x.shape)
print('train_y:', train_y.shape)
print('test_x:', test_x.shape)
print('test_y:', test_y.shape)

2017-08-15 20:58:40,996 [hybrid-lstm.tool | INFO] Load existing dataset


data_x: (43642, 7)
data_y: (43642, 1)
train_x: (34590, 32, 7)
train_y: (34590, 32, 1)
test_x: (8648, 32, 7)
test_y: (8989, 32, 1)


# Models

In [16]:
model = Sequential()
model.add(LSTM(64, batch_input_shape=(None, BATCH, 7)))